In [1]:
" Using LSTM and tensorflow for end of month  balance prediction . Used 5 timesteps and 13 features  "

%reset -f
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import numpy as np
import tensorflow as tf  
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import train_test_split
import os
import time
import functools

In [11]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(df1, df1['m6'], test_size=0.3, random_state=2018)
print("Training :" ,X_train_lstm.shape) 
print("Testing  :" ,X_test_lstm.shape)
 
X_train  =  X_train_lstm.copy()
X_test   =  X_test_lstm.copy()     
X_train[log_list]  = X_train_lstm[log_list].apply(lambda x: np.sign(x)*(np.log(np.abs(x) + 1)))
X_test[log_list]   = X_test_lstm[log_list].apply(lambda x: np.sign(x)*(np.log(np.abs(x) + 1)))
y_train  = y_train_lstm.apply(lambda x: np.sign(x)*(np.log(np.abs(x) + 1)))
y_test   = y_test_lstm.apply(lambda x: np.sign(x)*(np.log(np.abs(x) + 1)))



Training : (1166, 78)
Testing  : (501, 78)


C:\Users\Prasanna\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sign
  if __name__ == '__main__':


In [12]:
dv=y_train.values
lstm_cols=[cols for cols  in X_train.columns if 'eom' in cols and 'm1' not in cols]
lstm_op=X_train[lstm_cols]
train_cols=[cols for cols in X_train.columns if 'm6' not in cols]
 
train_data=X_train[train_cols]


In [14]:
scaler_train=StandardScaler()
train_data=pd.DataFrame(scaler_train.fit_transform(train_data), columns =train_data.columns.tolist())
 

In [15]:
scaler_op=StandardScaler()
mean_std_op=scaler_op.fit(lstm_op)
lstm_op=pd.DataFrame(scaler_op.fit_transform(lstm_op), columns =lstm_op.columns.tolist())
mean_std_op.mean_

array([8.67437082, 8.67890725, 8.70775032, 8.67624219, 8.6047824 ])

In [35]:
def get_batches(train_data,lstm_op, batch_size, seq_length ,num_inputs,num_outputs):
    
    
    x1=train_data.values 
    y1=lstm_op.values
    
    # Get the number of characters per batch and number of batches we can make
    num_records = batch_size * seq_length
    n_batches = len(x1)*seq_length//num_records
    

    #print("n_batches" ,n_batches)
    
    # Keep only enough characters to make full batches
    x1 = x1[:n_batches * batch_size]
    y1 = y1[:n_batches * batch_size]
 
    #print(x1.shape)    
    # Reshape into batch_size rows
    x1 = x1.reshape((batch_size, -1))
    y1 = y1.reshape((batch_size, -1))
    
    for n in range(0, x1.shape[1], seq_length):
        # The features
        x = x1[:, n:n+seq_length]
        # The targets
        stop =  int(seq_length/num_inputs)
        start = int(n/num_inputs)
        y = y1[:, start:start+stop]
        
        # For the very last batch, y will be one character short at the end of 
        # the sequences which breaks things. To get around this, I'll make an 
        # array of the appropriate size first, of all zeros, then add the targets.
        # This will introduce a small artifact in the last batch, but it won't matter.
         
        #x=x.reshape(batch_size,n_steps,num_inputs)
        #y=y.reshape(-1,num_inputs)
        #print(int(seq_length /num_inputs))
        x=x.reshape(batch_size,int(seq_length /num_inputs),num_inputs)
        y=y.reshape(-1,num_outputs)
        yield x, y
    
 

In [36]:
def build_inputs(batch_size, num_steps,num_inputs,num_outputs):
       # Declare placeholders we'll feed into the graph


    inputs = tf.placeholder(tf.float32, [None, num_steps,num_inputs], name='inputs')
    targets = tf.placeholder(tf.float32, [None, num_outputs], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [37]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
       
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state
""" 
 tf.nn.rnn_cell.MultiRNNCell, which takes a list of RNNCells as its inputs and wraps them into a single cell:

cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicRNNCell(state_size)] * num_layers)
"""

In [38]:
def build_output(lstm_output, lstm_size, out_size):
   
    seq_output = tf.concat(lstm_output, axis=1)
     
    op = tf.reshape(seq_output, [-1, lstm_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('matmul'):
        matmul_w = tf.Variable(tf.truncated_normal((lstm_size, out_size), stddev=0.1))
        matmul_b =  tf.Variable(tf.zeros(out_size)) 
        #tf.Variable(tf.truncated_normal(out_size,stddev=.1))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    prediction = tf.matmul(op, matmul_w) + matmul_b
     
    return (prediction)

In [39]:
def build_loss_optimizer(targets,prediction,learning_rate):
    #targets1=tf.reshape(targets, [batch_size,num_steps,-1])
    #targets1=tf.reshape(targets1, [-1,1])
    
    loss = tf.reduce_mean(tf.square(prediction - targets))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return(loss,optimizer)

In [40]:
class bal_lstm:
    
    def __init__(self, out_size=1, 
                       batch_size=64,  
                       num_steps=50,
                       lstm_size=128, 
                       num_layers=2, seq_length=5,
                       learning_rate=0.001,num_inputs=1,num_outputs=1 
                       ):
           
        tf.reset_default_graph()        
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps,num_inputs,num_outputs)
        
        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        self.lstm_outputs, state = tf.nn.dynamic_rnn(cell,self.inputs,initial_state=self.initial_state)
        self.final_state = state
        
        #Get   predictions  
        self.prediction  = build_output(self.lstm_outputs, lstm_size, out_size)
        
        # Loss and optimizer (with gradient clipping)
        self.loss ,self.optimizer = build_loss_optimizer(self.targets,self.prediction,learning_rate)
         

In [49]:
counter=0
batch_size=100 # number of records /batch
num_steps=5   # number of sequence steps per batch
num_inputs=13 # number of features
seq_length=num_inputs * num_steps
num_outputs=1
for x, y in get_batches(train_data,lstm_op, batch_size, seq_length,num_inputs,num_outputs):
    counter+=1
    #print("x",x)
    #print("y",y)
    
print("Total number of batches ::", counter)    
print(x.shape)
print(x[0:3])




Total number of batches :: 11
(100, 5, 13)
[[[ 1.13829763 -0.18723929 -0.09613529  0.14970446 -0.00981461
    4.25408392  0.60075279 -0.50982402  2.07447023  5.02754843
   -0.07130663  0.13224163  1.12751989]
  [ 1.87602168 -0.14186122 -0.02599943  0.13267013  0.12193032
    1.55671379  0.88127277 -0.14644495  1.98268884  6.63335671
   -0.17983412  0.13552211  1.23105918]
  [ 1.2458321  -0.11874051 -0.02343834  0.11005274  0.03530104
    3.38346554  0.94276045 -0.4261717   0.29892846  4.76564991
    0.16754538  0.13652094  1.26657066]
  [ 0.7863725  -0.09509291  0.47197749  0.11176771  0.0601339
    1.58156719  1.09199648 -0.64155495  0.33750265  4.43724489
   -0.16478629  0.13580928  1.27268899]
  [ 3.77116263 -0.17886996 -0.08718766 -0.48669569 -2.59451067
    2.66362051  0.64913983 -0.42295495  0.42712468  6.27067953
   -0.18458534  0.15154847  1.07539688]]

 [[ 0.59985745 -0.18723929 -0.6510548   0.14970446 -0.23674757
    0.00851656  0.19039023 -0.64154456 -0.1971068  -0.33701022


In [50]:
epochs=900
seq_length = num_inputs * num_steps # sequence length
lstm_size  = 128      # size of hidden layers in LSTMs
num_layers = 6        #Number of LSTM layers
#learning_rate = 0.001  
learning_rate=.0001    #Learning rate
keep_prob     = 0.8      #Dropout keep probability
out_size=1
 
 

In [51]:
epochs =100
np.random.seed(10)
print_every_n = 100

# Save every N iterations
saver_every_n = 100
 
model = bal_lstm(out_size=out_size, 
                batch_size=batch_size, 
                num_steps=num_steps,
                lstm_size=lstm_size,
                num_layers=num_layers,seq_length=seq_length, 
                learning_rate=learning_rate,num_inputs=num_inputs,num_outputs=num_outputs)
saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    for e in range(epochs):
        #print("sri krishna")
        for x, y in get_batches(train_data,lstm_op, batch_size, seq_length,num_inputs,num_outputs):
                                 
            # Train network
            new_state = sess.run(model.initial_state)
             
            loss = 0
            counter += 1
            #print("counter",counter)
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state} 
            
             
            batch_loss, new_state, predictions_train,_ = sess.run([model.loss, 
                                                 model.final_state,
                                                 model.prediction, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            if (counter % print_every_n == 0):
                end = time.time()
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.9f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
                 
            if (counter % saver_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))   
    
       
     
             
     
            #pred_train.append(predictions_test)
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))  
     
     


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch: 10/900...  Training Step: 100...  Training loss: 0.621169031...  0.1299 sec/batch
Epoch: 19/900...  Training Step: 200...  Training loss: 0.760043204...  0.1509 sec/batch
Epoch: 28/900...  Training Step: 300..

In [39]:
## test test data
batch_size =test_data.shape[0]
pred_test=[]
model =   bal_lstm(out_size=out_size, 
                  batch_size=batch_size, 
                  num_steps=num_steps,
                  lstm_size=lstm_size,
                  num_layers=num_layers, 
                  learning_rate=learning_rate,num_inputs=num_inputs,num_outputs=num_outputs)
saver = tf.train.Saver()
test_acc = []
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    for x, y in get_batches(test_data,lstm_op_test, batch_size, seq_length,num_inputs,num_outputs):
         
        new_state = sess.run(model.initial_state)
        feed_train = {model.inputs: x,
                        model.keep_prob: 1 ,
                        model.initial_state: new_state}
        predictions_test,new_state = sess.run([model.prediction,model.final_state],
                                feed_dict=feed_train )
        for n in range(0, predictions_test.ravel().shape[0],num_steps):
            pred_test.append(predictions_test.ravel()[n+num_steps-1])
        #print(predictions_train.ravel())    
         

INFO:tensorflow:Restoring parameters from checkpoints\i500_l512.ckpt


In [40]:
pred_test1= [(cols*mean_std_op_test.scale_[4] + mean_std_op_test.mean_[4] ) for cols in pred_test]
pred_test2= [np.exp(cols) for cols in pred_test1]

 
 